In [1]:
from Read_data import ReadINCAFile
from BaseBlocks import *
from BaseClasses import Block, Connection
from SimulationClass import Simulation
from SimulinkBlockClass import SimulinkBlock

In [ ]:
class AccumCtr(SimulinkBlock):
    def __init__(self, name, entry, Tol, I, DevMax, r, MxL, ValT, Ctr, Val):
        super().__init__(name, [entry, Tol, I, DevMax, r, MxL, ValT], [Ctr, Val])

        self.node = [{'data':{'id':name,'label':'AccumCtr'}}] \
            + [{'data': {'id': self.name + '_' + i, 'parent': self.name,'label':i}} for i in self.inputs_list] \
            + [{'data': {'id': self.name + '_' + i, 'parent': self.name,'label':i}} for i in self.outputs_list]

        self.addEntryCondition(EntryCondition(name='entry',entrySignal=entry))
        
        self.addBlock(Min(name='min1',signal1='delayedCtr',signal2=Tol,output='min1'))
        
        self.addBlock(Constant(name='Constant1',value=0))
        self.addBlock(Max(name='max1',signal1=I,signal2='Constant1',output='max1'))

        self.addBlock(Subtract(name='sub1',pos='max1',neg='min1',output='sub1'))

        self.addBlock(Min(name='min2',signal1='sub1',signal2=DevMax,output='min2'))

        self.addBlock(Add(name='add1',pos1='delayedCtr',pos2='min2',output='add1'))

        self.addBlock(Constant(name='Constant3',value=0))
        self.addBlock(Max(name='max3',signal1='add1',signal2='Constant3',output='max3'))

        self.addBlock(Min(name='min3',signal1='max3',signal2='div1',output='min3'))

        self.addBlock(Constant(name='Constant4',value=0))
        self.addBlock(Switch(name='switchCtr',switched='Constant4',default='min3',condition=r,output='ctr1'))

        self.addBlock(Delay(name='delayCtr',signalIn='ctr1',signalOut='delayedCtr'))
        
        self.addBlock(Multiply(name='mul1',factor1='ctr1',factor2='ts',output=Ctr))

        self.addBlock(SampleTime(name='ts',value=0.04))
        self.addBlock(Divide(name='div1',numerator=MxL,denominator='ts', output='div1'))

        self.addBlock(Add(name='add2',pos1='ts',pos2='delayVal',output='add2'))

        self.addBlock(Constant(name='Constant5',value=0))
        self.addBlock(Switch(name='switchVal',switched='Constant5',default='add2',condition=r,output='switchVal'))
        
        self.addBlock(Min(name='min4',signal1='switchVal',signal2=ValT,output='min4'))
        self.addBlock(Delay(name='delayVal',signalIn='min4',signalOut='delayVal'))

        self.addBlock(GreaterThanOrEqual(name='geq',first='switchVal',second=ValT,output=Val))

        self.compileBlock()

In [3]:
data = ReadINCAFile('vnt_data/',['rVcAesVntPosnM_X_DiffPosn','xVcAesVntPosnM_B_Entry','rVcAesVntPosnM_X_AccuDev'])
data['yVcObdCmn_B_ObdInitReq'] = 0
data = data.where((data['rVcAesVntPosnM_X_DiffPosn'].diff() != 0) & (data['xVcAesVntPosnM_B_Entry'] == 1)).dropna(ignore_index=True)
data

100%|██████████| 1/1 [00:04<00:00,  4.11s/it]


,rVcAesVntPosnM_X_DiffPosn,xVcAesVntPosnM_B_Entry,rVcAesVntPosnM_X_AccuDev,yVcObdCmn_B_ObdInitReq
0,0.220139,1.0,0.008806,0.0
1,0.179375,1.0,0.007175,0.0
2,0.138603,1.0,0.005544,0.0
3,0.179375,1.0,0.007175,0.0
4,0.138603,1.0,0.005544,0.0
...,...,...,...,...
132816,2.468430,1.0,0.098737,0.0
132817,4.257301,1.0,0.170292,0.0
132818,2.396923,1.0,0.095877,0.0
132819,10.358604,1.0,0.295877,0.0


In [4]:
block = SimulinkBlock(name='Nested block',inputs=['rVcAesVntPosnM_X_DiffPosn','xVcAesVntPosnM_B_Entry','yVcObdCmn_B_ObdInitReq'], outputs=['X_Accudev','B_EvalVld'])

block.addCalibrationParameter(Parameter(name='X_DiffTol'))
block.addCalibrationParameter(Parameter(name='X_DevMax'))
block.addCalibrationParameter(Parameter(name='t_MaxL'))
block.addCalibrationParameter(Parameter(name='t_Val'))

block.addBlock(AccumCtr(name='AccumCtr',entry='xVcAesVntPosnM_B_Entry',Tol='X_DiffTol',I='rVcAesVntPosnM_X_DiffPosn',DevMax='X_DevMax',r='yVcObdCmn_B_ObdInitReq',MxL='t_MaxL',ValT='t_Val',Ctr='X_Accudev',Val='B_EvalVld'))

block.compileBlock()

simulation = Simulation(sim=block,data=data)
simulation.setCalibration(X_DiffTol=np.arange(1,6),
                          X_DevMax=np.arange(1,6),
                          t_MaxL=[80],
                          t_Val=np.arange(5,7))

AccumCtr compiled successfully!
Nested block compiled successfully!
X_DiffTol calibrated to [1 2 3 4 5]
X_DevMax calibrated to [1 2 3 4 5]
t_MaxL calibrated to [80]
t_Val calibrated to [5 6]


In [5]:
simulation.runSimulation()

132821it [00:23, 5742.26it/s]


In [7]:
simulation.showMax('X_Accudev')